In [1]:
import numpy as np
import pandas as pd

DATA = '/kaggle/input/exoplanet-dataset-scraped-from-science-nasa-gov/exoplanet_data.csv'

df = pd.read_csv(filepath_or_buffer=DATA).replace('Unknown', np.nan).dropna()
df[['Light_Years_From_Earth', 'Stellar_Magnitude']] = df[['Light_Years_From_Earth', 'Stellar_Magnitude']].astype(float)
df.head()

,all_planet_names,Light_Years_From_Earth,Planet_Mass,Stellar_Magnitude,Discovery_Date
0,TOI-4559 b,31.6354,2.59 Earths,13.115,2024
1,TOI-1739 b,70.9819,3.52 Earths,10.692,2024
2,Kepler-1660 AB b,1188.5400,4.992 Jupiters,13.595,2023
4,TIC 139270665 c,189.8750,4.89 Jupiters,10.385,2024
5,TIC 139270665 b,189.8750,0.463 Jupiters,10.385,2024


In [2]:
df.shape

(5372, 5)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5372 entries, 0 to 5625
Data columns (total 5 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   all_planet_names        5372 non-null   object 
 1   Light_Years_From_Earth  5372 non-null   float64
 2   Planet_Mass             5372 non-null   object 
 3   Stellar_Magnitude       5372 non-null   float64
 4   Discovery_Date          5372 non-null   int64  
dtypes: float64(2), int64(1), object(2)
memory usage: 251.8+ KB


Let's plot the star magnitude vs. the distance from Earth. We will act like the magnitude is a function of the distance from Earth, which is not really true.

In [4]:
from plotly import express

express.scatter(data_frame=df, x='Light_Years_From_Earth', y='Stellar_Magnitude', log_x=True, hover_name='all_planet_names')